In [184]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep
import re

In [185]:
# url of mixed dog breeds
url = 'https://www.dogbreedinfo.com/hybriddogs.htm'

In [186]:
res = requests.get(url)
res.status_code

200

In [187]:
# soup of all the <li> of dog breeds
soup = BeautifulSoup(res.content, 'lxml')
list_items = soup.find('div', {'class': 'container'}).find_all('li')

# Extract dog name and breeds from soup

In [188]:
# https://stackoverflow.com/questions/6116978/how-to-replace-multiple-substrings-of-a-string
# Three lines of code below taken from stackoverflow, answered by Andrew Clark

# dictionary of things to replace
rep = {'(': '', ')':'', 'mix':'', ' - ':'', '\xa0': ' ', '-\t':'', '- ':'', '= ': ''}
# this code does some stuff and works! 
# It goes along with the "pattern.sub(lambda m: rep[re.escape(m.group(0))]" used in the loop
rep = dict((re.escape(k), v) for k, v in rep.items()) 
pattern = re.compile("|".join(rep.keys()))


# initialize empty list to add dogs to
dogs = []

# iterates over lists of dog names and breeds
for li in list_items[5:]:

    mutt = {}

    # breaks out of lists of dogs at the end of the loop
    if li.text == 'Mixed Breed Dog Information':
        break

    # if the <li> has an <a> tag
    if li.find('a'):

        # cleans the name of the dog
        # regex pattern to sub characters from breeds. (parenthesis, the word 'mix', random chars, etc)
        name = li.find('a').text.lower().strip()
        breeds = pattern.sub(lambda m: rep[re.escape(m.group(0))], li.contents[-1]).lower().strip().split(' x ')

        # assigns name and breeds to mutt dict
        mutt['name'] = name
        mutt['breeds'] = [breed.strip() for breed in breeds]

    # if there is no <a> in the <li>
    else:
    
        # cleans the name and breeds to be able to split. ".replace" chains to make all input usuable
        cleaned_name = li.text.lower().replace(' -', ' - ').replace('- ', ' - ').replace('\xa0',' ').split(' - ')[0].strip()
        cleaned_breed = li.text.lower().replace(' -', ' - ').replace('- ', ' - ').replace('\xa0',' ').split(' - ')[1].strip()

        # regex pattern to sub characters from name and breeds. (parenthesis, the word 'mix', random chars, etc)
        name = pattern.sub(lambda m: rep[re.escape(m.group(0))], cleaned_name).strip()
        breeds = pattern.sub(lambda m: rep[re.escape(m.group(0))], cleaned_breed).strip().split(' x ')

        # assigns name and breeds to mutt dict
        mutt['name'] = name
        mutt['breeds'] = [breed.strip() for breed in breeds]

    # appends dog name and breeds to dogs list
    dogs.append(mutt)
    # print(f"{name} added.")

# Fixing breed names

In [189]:
breeds_to_fix = []
names = []

# iterates over all the dogs
for dog in dogs:
    
    # iterates over all the breeds for each dog
    for breed in dog['breeds']:

        # checks for weirdness in the breeds and assigns that dog to a list to be dealt with. BAD DOG!
        if ' with ' in breed:
            names.append(dog['name'])
            breeds_to_fix.append(dog)
        elif ' or ' in breed:
            names.append(dog['name'])
            breeds_to_fix.append(dog)
        elif ' the ' in breed:
            names.append(dog['name'])
            breeds_to_fix.append(dog)
        elif ' ed ' in breed:
            names.append(dog['name'])
            breeds_to_fix.append(dog)
        elif ' Miniature ' in breed:
            names.append(dog['name'])
            breeds_to_fix.append(dog)
        elif ' Toy ' in breed:
            names.append(dog['name'])
            breeds_to_fix.append(dog)
        elif ' Mini ' in breed:
            names.append(dog['name'])
            breeds_to_fix.append(dog)

In [190]:
# removes all the bad dogs
for dog in dogs:
    if dog['name'] in names:
        dogs.remove(dog)

In [191]:
breeds_to_fix[0]

{'name': 'aussie-corgi',
 'breeds': ['australian shepherd or miniature australian shepherd',
  'pembroke welch corgi']}

In [192]:
dogs.append(
    {
        "name": "aussie-corgi",
        "breeds": ["australian shepherd", "pembroke welch corgi",],
    }
)

dogs.append(
    {
        "name": "aussie-corgi",
        "breeds": ["miniature australian shepherd", "pembroke welch corgi",],
    }
)

In [193]:
breeds_to_fix[1]

{'name': 'aussie pom',
 'breeds': ['australian shepherd or mini or toy australian shepherd',
  'pomeranian']}

In [194]:
dogs.append(
    {"name": "aussie pom", "breeds": ["australian shepherd", "pomeranian"],}
)

dogs.append(
    {"name": "aussie pom", "breeds": ["toy australian shepherd", "pomeranian"],}
)

dogs.append(
    {"name": "aussie pom", "breeds": ["mini australian shepherd", "pomeranian"],}
)


In [195]:
breeds_to_fix[2]

{'name': 'austi-pap',
 'breeds': ['australian shepherd standard, toy or miniature', 'papillon']}

In [196]:
dogs.append(
    {"name": "austi-pap", "breeds": ["australian shepherd", "papillon"],}
)

dogs.append(
    {"name": "austi-pap", "breeds": ["miniature australian shepherd", "papillon"],}
)

dogs.append(
    {"name": "austi-pap", "breeds": ["toy australian shepherd", "papillon"],}
)

In [197]:
breeds_to_fix[3]

{'name': 'beagi', 'breeds': ['beagle', 'pembroke or cardigan welsh corgi']}

In [198]:
dogs.append({"name": "beagi", "breeds": ["beagle", "pembroke"]})

dogs.append({"name": "beagi", "breeds": ["beagle", "cardigan welsh corgi"]})

In [199]:
breeds_to_fix[4]

{'name': 'border stack',
 'breeds': ['border collie ed with jack russell terrier',
  'staffordshire bull terrier']}

In [200]:
# what to do here?
{
    "name": "border stack",
    "breeds": [
        "border collie ed with jack russell terrier",
        "staffordshire bull terrier",
    ],
}

{
    "name": "border stack",
    "breeds": [
        "border collie ed with jack russell terrier",
        "staffordshire bull terrier",
    ],
}

{'name': 'border stack',
 'breeds': ['border collie ed with jack russell terrier',
  'staffordshire bull terrier']}

In [201]:
breeds_to_fix[5]

{'name': 'bully pitsky',
 'breeds': ['american bully', 'siberian husky or alaskan husky']}

In [202]:
dogs.append(
    {"name": "bully pitsky", "breeds": ["american bully", "siberian husky"],}
)

dogs.append(
    {"name": "bully pitsky", "breeds": ["american bully", "alaskan husky"],}
)

In [203]:
breeds_to_fix[6]

{'name': 'cotralian',
 'breeds': ['cocker spaniel',
  'standard, miniature or toy australian shepherd']}

In [204]:
dogs.append(
    {"name": "cotralian", "breeds": ["cocker spaniel", "australian shepherd"],}
)

dogs.append(
    {"name": "cotralian", "breeds": ["cocker spaniel", "toy australian shepherd"],}
)

dogs.append(
    {
        "name": "cotralian",
        "breeds": ["cocker spaniel", "miniature australian shepherd"],
    }
)

In [205]:
breeds_to_fix[7]

{'name': 'english cotralian',
 'breeds': ['english cocker spaniel',
  'standard, miniature or toy australian shepherd']}

In [206]:
dogs.append(
    {
        "name": "english cotralian",
        "breeds": ["english cocker spaniel", "australian shepherd",],
    }
)

dogs.append(
    {
        "name": "english cotralian",
        "breeds": ["english cocker spaniel", "toy australian shepherd",],
    }
)

dogs.append(
    {
        "name": "english cotralian",
        "breeds": ["english cocker spaniel", "miniature australian shepherd",],
    }
)

In [207]:
breeds_to_fix[8]

{'name': 'labradoodle miniature',
 'breeds': ['labrador retriever', 'toy or miniature poodle']}

In [208]:
dogs.append(
    {
        "name": "labradoodle miniature",
        "breeds": ["labrador retriever", "miniature poodle"],
    }
)

dogs.append(
    {"name": "labradoodle miniature", "breeds": ["labrador retriever", "toy poodle"],}
)

In [209]:
breeds_to_fix[9]

{'name': 'miniature goldendoodle',
 'breeds': ['golden retriever', 'toy or miniature poodle']}

In [210]:
dogs.append({
    "name": "miniature goldendoodle",
    "breeds": ["golden retriever", "miniature poodle"],
})

dogs.append({
    "name": "miniature goldendoodle",
    "breeds": ["golden retriever", "toy poodle"],
})

In [211]:
breeds_to_fix[10]

{'name': 'miniature labradoodle',
 'breeds': ['labrador retriever', 'toy or miniature poodle']}

In [212]:
dogs.append(
    {
        "name": "miniature labradoodle",
        "breeds": ["labrador retriever", "miniature poodle"],
    }
)

dogs.append(
    {"name": "miniature labradoodle", "breeds": ["labrador retriever", "toy poodle"],}
)

In [213]:
breeds_to_fix[11]

{'name': 'old anglican bulldogge',
 'breeds': ['american pit bull terrier or american staffordshire terrier',
  'bulldog']}

In [214]:
dogs.append(
    {
        "name": "old anglican bulldogge",
        "breeds": ["american pit bull terrier", "bulldog",],
    }
)

dogs.append(
    {
        "name": "old anglican bulldogge",
        "breeds": ["american staffordshire terrier", "bulldog",],
    }
)

In [215]:
breeds_to_fix[12]

{'name': 'pitsky',
 'breeds': ['american pitbull terrier', 'siberian husky or alaskan husky']}

In [216]:
dogs.append({"name": "pitsky", "breeds": ["american pitbull terrier", "alaskan husky"]})

dogs.append({"name": "pitsky", "breeds": ["american pitbull terrier", "siberian husky"]})

In [217]:
breeds_to_fix[13]

{'name': 'shel-aussie',
 'breeds': ['australian shepherd or miniature australian shepherd',
  'shetland sheepdog']}

In [218]:
dogs.append(
    {"name": "shel-aussie", "breeds": ["australian shepherd", "shetland sheepdog",],}
)

dogs.append(
    {
        "name": "shel-aussie",
        "breeds": ["miniature australian shepherd", "shetland sheepdog",],
    }
)

In [219]:
breeds_to_fix[14]

{'name': 'sibercaan',
 'breeds': ['siberian husky or native american indian dog', 'canaan dog']}

In [220]:
dogs.append({"name": "sibercaan", "breeds": ["siberian husky", "canaan dog"]})

dogs.append(
    {"name": "sibercaan", "breeds": ["native american indian dog", "canaan dog"]}
)

In [221]:
df = pd.DataFrame(dogs)
df.head()

,name,breeds
0,afador,"[afghan hound, labrador retriever]"
1,afaird,"[afghan hound, briard]"
2,affen spaniel,"[affenpinscher, cocker spaniel]"
3,affen border terrier,"[affenpinscher, border terrier]"
4,affen tzu,"[affenpinscher, shih tzu]"


In [222]:
df = df[df['name'] != 'minicoonhound']

In [223]:
df.to_csv('./datasets/mutts.csv', index=False)